# 4. Magnitude Analysis: Comparing Genotypes

This notebook is for fitting the growth models to globally scaled data. The goal of this analysis is to preserve the relative size differences between plants, allowing us to identify high- and low-growth genotypes.

The workflow is:
1. Load the full dataset.
2. Apply a single **global scaler** to the entire `area` column.
3. Loop through each plant, fitting the models to this globally-scaled data.
4. Save the results to a new CSV file for comparison.

In [6]:
import pandas as pd
import numpy as np
import sys
import os
import joblib
from scipy.optimize import minimize
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm

# Add parent directory to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import custom functions
from src.data_processing import load_data, preprocess_data
from src.models import linear, ricker_model, logistic_fun, exponential, gen_vb
from src.fitting import (cost_linear, cost_ricker, cost_logistic, cost_exponential, cost_gvb,
                         AIC_linear, AIC_ricker, AIC_logistic, AIC_exp, AIC_gen_vb)
from src.analysis import calculate_aic_weights

In [7]:
# --- Load Data ---
FEATURES_PATH = '../data/5.25.2024_CBI.0010_York_Poplar_RGB.side_features.csv'
BARCODE_PATH = '../data/barcodes--5.25.2024_CBI.0010_York_Poplar.csv'
data_df, barcode_df = load_data(FEATURES_PATH, BARCODE_PATH)
merged_df = preprocess_data(data_df, barcode_df)

# --- Apply Global Scaler ---
# This is the key step for this analysis path
global_scaler = MinMaxScaler()
merged_df['area_globally_scaled'] = global_scaler.fit_transform(merged_df[['area']])

print("Global scaling applied successfully.")
merged_df[['area', 'area_globally_scaled']].describe()

Global scaling applied successfully.


,area,area_globally_scaled
count,3.407000e+03,3407.000000
mean,4.671095e+05,0.208544
std,2.888058e+05,0.130972
min,7.249000e+03,0.000000
25%,2.494235e+05,0.109825
50%,4.158690e+05,0.185307
75%,6.159930e+05,0.276061
max,2.212352e+06,1.000000


In [8]:
# Load the initial guesses from Notebook 02
PARAMS_PATH = '../results/data/global_optimal_params.joblib'
global_params = joblib.load(PARAMS_PATH)

q0_linear = global_params['linear']
q0_logistic = global_params['logistic']
q0_ricker = global_params['ricker']
q0_exp = global_params['exponential']
q0_gvb = global_params['gen_vb']

# Define parameter bounds
bounds_linear = ((0, 1), (0, 1))
bounds_logistic = ((0, 1), (0, 1), (0, 1.5))
bounds_ricker = ((0, 1), (0, 1), (0, 1.5))
bounds_exp = ((0, 1), (0, 1))
bounds_gvb = ((0, 5), (0.01, 2), (0.01, 0.99), (0.8, 1.5))

# --- Main Loop ---
results_list = []
unique_plants = merged_df['Plant Info'].unique()

for plant_id in tqdm(unique_plants, desc="Fitting models (Global Scale)"):
    plant_data = merged_df[merged_df['Plant Info'] == plant_id]
    if len(plant_data) < 5: continue

    dd = plant_data["Days_Since_2024_05_26"].to_numpy()
    area = plant_data["area_globally_scaled"].to_numpy()

    result_row = {'Plant Info': plant_id, 'Plant Genotype': plant_data['Plant.Genotype'].iloc[0]}

    # --- Fit Models AND Calculate AIC ---
    # 1. Linear
    lin_opt = minimize(cost_linear, q0_linear, args=(dd, area), method='L-BFGS-B', bounds=bounds_linear)
    result_row['m0_optimal_linear'], result_row['k_optimal_linear'] = lin_opt.x
    result_row['AIC_Linear'] = AIC_linear(lin_opt.x, dd, area)

    # 2. Ricker
    ricker_opt = minimize(cost_ricker, q0_ricker, args=(dd, area), method='L-BFGS-B', bounds=bounds_ricker)
    result_row['W0_optimal_ricker'], result_row['kg_optimal_ricker'], result_row['m_optimal_ricker'] = ricker_opt.x
    result_row['AIC_Ricker'] = AIC_ricker(ricker_opt.x, dd, area)

    # 3. Logistic
    log_opt = minimize(cost_logistic, q0_logistic, args=(dd, area), method='L-BFGS-B', bounds=bounds_logistic)
    result_row['P0_optimal_log'], result_row['r_optimal_log'], result_row['K_optimal_log'] = log_opt.x
    result_row['AIC_Logistic'] = AIC_logistic(log_opt.x, dd, area)
    
    # 4. Exponential
    exp_opt = minimize(cost_exponential, q0_exp, args=(dd, area), method='L-BFGS-B', bounds=bounds_exp)
    result_row['m0_optimal_exp'], result_row['k_optimal_exp'] = exp_opt.x
    result_row['AIC_Exponential'] = AIC_exp(exp_opt.x, dd, area)

    # 5. Gen. VB
    gvb_opt = minimize(cost_gvb, q0_gvb, args=(dd, area), method='L-BFGS-B', bounds=bounds_gvb)
    result_row['m0_optimal_gvb'], result_row['k_optimal_gvb'], result_row['f_optimal_gvb'], result_row['A_optimal_gvb'] = gvb_opt.x
    result_row['AIC_Gen_VB'] = AIC_gen_vb(gvb_opt.x, dd, area)
    
    results_list.append(result_row)

print("Finished fitting models to all plants.")

Fitting models (Global Scale):   0%|          | 0/446 [00:00<?, ?it/s]

c:\Users\katie\OneDrive - East Tennessee State University\School\Data Saplings Organized\src\models.py:22: RuntimeWarning: divide by zero encountered in scalar divide
  P = K/(((K - P0)/P0)*np.exp(-r*t) + 1)
c:\Users\katie\OneDrive - East Tennessee State University\School\Data Saplings Organized\src\models.py:22: RuntimeWarning: invalid value encountered in divide
  P = K/(((K - P0)/P0)*np.exp(-r*t) + 1)
c:\Users\katie\OneDrive - East Tennessee State University\School\Data Saplings Organized\src\models.py:22: RuntimeWarning: invalid value encountered in scalar divide
  P = K/(((K - P0)/P0)*np.exp(-r*t) + 1)


Finished fitting models to all plants.


In [9]:
# Convert list to DataFrame
results_df_global = pd.DataFrame(results_list)

# Calculate AIC weights to compare models
results_df_global = calculate_aic_weights(results_df_global)

# Save the results to a NEW CSV file
RESULTS_SAVE_PATH = '../results/data/all_plant_model_results_GLOBAL.csv'
results_df_global.to_csv(RESULTS_SAVE_PATH, index=False)

print(f"Global results table shape: {results_df_global.shape}")
print(f"Results saved to: {RESULTS_SAVE_PATH}")
results_df_global.head()

Global results table shape: (446, 26)
Results saved to: ../results/data/all_plant_model_results_GLOBAL.csv


,Plant Info,Plant Genotype,m0_optimal_linear,k_optimal_linear,AIC_Linear,W0_optimal_ricker,kg_optimal_ricker,m_optimal_ricker,AIC_Ricker,P0_optimal_log,...,m0_optimal_gvb,k_optimal_gvb,f_optimal_gvb,A_optimal_gvb,AIC_Gen_VB,Linear_AIC_Weight,Ricker_AIC_Weight,Logistic_AIC_Weight,Exponential_AIC_Weight,Gen_VB_AIC_Weight
0,22_BESC-1126_2,BESC-1126,0.046448,0.004459,-69.792968,0.043244,0.082518,1.466149,-56.915927,0.045718,...,0.045171,0.041134,0.267598,0.832968,-57.172271,0.510839,8.167147e-04,0.479296,0.008119,0.000928
1,23_BESC-1207_2,BESC-1207,0.001673,0.000000,-108.238717,0.001673,0.000015,1.029286,-101.025409,0.001672,...,0.001662,0.010002,0.010003,0.800000,-88.648375,0.487505,1.323212e-02,0.013272,0.485964,0.000027
2,24_BESC-317_2,BESC-317,0.036432,0.007592,-58.837783,0.057779,0.091285,1.469024,-38.915053,0.051259,...,0.048339,0.024865,0.380790,0.837554,-47.207968,0.606117,2.860162e-05,0.154408,0.237638,0.001808
3,25_GW-9919_3,GW-9919,0.131187,0.011911,-52.042131,0.121604,0.078887,1.449028,-42.787962,0.125438,...,0.123911,0.048293,0.183367,0.883444,-39.789676,0.565840,5.535748e-03,0.418078,0.009310,0.001236
4,26_BESC-1068_5,BESC-1068,0.104920,0.014809,-40.440734,0.158736,0.049172,1.500000,-28.686603,0.132485,...,0.135176,0.010000,0.064956,0.987748,-38.878177,0.000237,6.641914e-07,0.006472,0.993182,0.000108
